In [1]:
import os
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")

In [3]:
LANGCHAIN_PROJECT

'langchain-agent'

In [4]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.0-pro")

/home/ravi0531rp/anaconda3/envs/langGraph-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from langchain_groq import ChatGroq
import os
llm=ChatGroq(model_name="Gemma2-9b-It")

## Simple AI Assistant


In [7]:
while True:
    question=input("type your question. if you want to quit the chat write quit")
    if question !="quit":
        print(llm.invoke(question).content)
    else:
        print("goodbye take care yourself")
        break

Hey there! 👋  What can I do for you today? 😊

Hello Ravi! It's nice to meet you. 

What can I do for you today?

As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to use it!

goodbye take care yourself


In [8]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [22]:
store={}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

config = {"configurable": {"session_id": "firstchat"}}
model_with_memory=RunnableWithMessageHistory(llm,get_session_history)


In [23]:
model_with_memory.invoke(("Hi! I'm ravi"),config=config).content


"Hello Ravi! 👋 \n\nIt's nice to meet you. What can I do for you today? 😄  \n\n"

In [24]:
model_with_memory.invoke(("What does my name mean?"),config=config).content


'That\'s a great question!  The name Ravi is of Sanskrit origin and has a beautiful meaning. \n\nIt means **"sun"** ☀️.  \n\nIn Hindu mythology, Ravi is the name of the sun god. So, your name symbolizes light, warmth, energy, and vitality!  ✨ \n\n\n\n'

In [25]:
import pprint


In [27]:
for k,v in store.items():
    print(k,v)

firstchat Human: Hi! I'm ravi
AI: Hello Ravi! 👋 

It's nice to meet you. What can I do for you today? 😄  


Human: What does my name mean?
AI: That's a great question!  The name Ravi is of Sanskrit origin and has a beautiful meaning. 

It means **"sun"** ☀️.  

In Hindu mythology, Ravi is the name of the sun god. So, your name symbolizes light, warmth, energy, and vitality!  ✨ 






## RAG with LCEL

In [28]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [29]:
### Reading the txt files from source directory

loader = DirectoryLoader('./', glob="./*.txt", loader_cls=TextLoader)
docs = loader.load()

### Creating Chunks using RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len
)
new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]

### Creating Retriever using Vector DB

db = Chroma.from_documents(new_docs, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 4})

In [31]:
doc_strings[:5]

['Llama (Large Language Model Meta AI) is a family',
 'a family of autoregressive large language models',
 'models released by Meta AI starting in February',
 'February 2023.[2][3] The latest version is Llama',
 'is Llama 3 released in April 2024.[4]']

In [32]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)

In [33]:
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n')

In [34]:
retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
    )

In [35]:
question ="what is llama3? can you highlight 3 important points?"
print(retrieval_chain.invoke(question))

Based on the provided documents, Llama3 is:

1. **An 8B parameter language model:** This refers to the size and complexity of the model. 
2. **Released in April 2024:** This indicates the timeframe of its public availability.
3. **Accessible outside of Meta:**  This means the model is available for use by developers and researchers beyond Meta's own platforms.  


Let me know if you have any other questions! 



## Tools and Agents

In [36]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [37]:
api_wrapper=WikipediaAPIWrapper()


In [38]:
tool=WikipediaQueryRun(api_wrapper=api_wrapper)


In [39]:
tool.name


'wikipedia'

In [40]:
tool.description


'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [41]:
tool.args


{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [42]:
print(tool.run({"query": "langchain"}))


Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

Page: Milvus (vector database)
Summary: Milvus is a distributed vector database developed by Zilliz. It is available as both open-source software and a cloud service.
Milvus is an open-source project under LF AI & Data Foundation distributed under the Apache License 2.0.



Page: Retrieval-augmented generation
Summary: Retrieval Augmented Generation (RAG) is a technique that grants generative artificial intelligence models information retrieval capabilities. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information to augment infor

In [43]:
from langchain_community.tools import YouTubeSearchTool


In [44]:
tool2=YouTubeSearchTool()


In [45]:
tool2.run("casetoo")


"['https://www.youtube.com/watch?v=tQL602cjfCY&pp=ygUHY2FzZXRvbw%3D%3D', 'https://www.youtube.com/watch?v=GRpG7HUwMkU&pp=ygUHY2FzZXRvbw%3D%3D']"

In [46]:
from langchain_community.tools.tavily_search import TavilySearchResults


In [47]:
tool3=TavilySearchResults()


In [48]:
tool.invoke({"query": "What happened in the latest T20 world cup"})


"Page: 2024 Men's T20 World Cup\nSummary: The 2024 ICC Men's T20 World Cup was the ninth edition of the ICC Men's T20 World Cup. It was co-hosted by the West Indies and the United States from 1 to 29 June 2024; the tournament was hosted by the West Indies for the second time, and it was also the first major ICC tournament to feature matches played in the United States.\nThe tournament expanded from 16 to 20 teams, including the two hosts, the top eight teams from the 2022 edition, the two highest-ranked teams in the ICC Men's T20I Team Rankings not already qualified, and eight other teams determined through regional qualifiers. Canada and Uganda qualified for the men's T20 World Cup for the first time, while the United States participated for the first time by virtue of being co-hosts.\nEngland were the defending champions and were beaten in the semi-finals by India, who went on to win their second T20 World Cup title, defeating South Africa by 7 runs and equalling England and West Ind

## Agents

In [49]:
from langchain.agents import AgentType
from langchain.agents import load_tools 
from langchain.agents import initialize_agent

In [59]:
# groq
llm=ChatGroq(model_name="llama3-groq-70b-8192-tool-use-preview")

In [60]:
tool=load_tools(["wikipedia"],llm=llm)

In [61]:
agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)


In [62]:
agent.run("What is current GDP of India?")




> Entering new AgentExecutor chain...
I can use Wikipedia to find the current GDP of India.

Action: wikipedia
Action Input: GDP of India
Observation: Page: Economy of India
Summary: The economy of India is a developing mixed economy with a notable public sector in strategic sectors. It is the world's fifth-largest economy by nominal GDP and the third-largest by purchasing power parity (PPP); on a per capita income basis, India ranked 141th by GDP (nominal) and 125th by GDP (PPP). From independence in 1947 until 1991, successive governments followed the Soviet model and promoted protectionist economic policies, with extensive Sovietization, state intervention, demand-side economics, natural resources, bureaucrat-driven enterprises and economic regulation. This is characterised as dirigism, in the form of the Licence Raj. The end of the Cold War and an acute balance of payments crisis in 1991 led to the adoption of a broad economic liberalisation in India and indicative planning. Sinc

KeyboardInterrupt: 

## Custom Tool

In [63]:
from langchain.agents import tool
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

In [65]:
get_word_length.args

{'word': {'title': 'Word', 'type': 'string'}}

In [66]:
get_word_length.description

'Returns the length of a word.'